# Machine Learning with PyTorch

## Natural Language Processing with AllenNLP

<font size="+1"><b><u>What is AllenNLP?</u></b></font>
<a href="AllenNLP_0.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">What is SpaCy?</font>
<a href="AllenNLP_1.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">High Level Interfaces to NLP using PyTorch</font>
<a href="AllenNLP_2.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">Sentiment Analysis</font>
<a href="AllenNLP_3.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">TBD</font> 
<a href="AllenNLP_4.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">TBD</font>
<a href="AllenNLP_5.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">TBD</font>
<a href="AllenNLP_6.ipynb"><img src="img/open-notebook.png" align="right"/></a>

AllenNLP is a library built on top of PyTorch that provides a large number of well tested, powerful, higher-level abstractions that are specifically useful for performing Natural Language Processing tasks with using neural networks.

It is much easier to develop and test these NLP models using these higher abstractions than with the lower-level PyTorch basics—even those in `torch.nn` and `torch.nn.functional`.  A lot of the "busy work with training schedules, metrics, optimization functions, data readers, and other aspects are wrapped in convenience functions or classes within AllenNLP.  However, using AllenNLP, you often make direct use of PyTorch functions, and under the hood all the computational work remains PyTorch. 

In [1]:
import warnings
warnings.simplefilter('ignore')

AllenNLP can be used to construct models in areas such as the following, each of which have [excellent demos](https://allennlp.org/models) at their website.  If you wish to use it so, the tools in AllenNLP are exposed as command line tools as well.  Behavior of models can be configured with JSON files in many cases rather than programmed in Python, giving (optionally) a more declarative format.  Actually, the configurations use a JSON superset called [Jsonnet](https://jsonnet.org/learning/tutorial.html) that can optionally use a few additional constructs such as local variables.

### Machine Comprehension

In Python, using a pre-trained model:

In [2]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/"
                                "bidaf-model-2017.09.15-charpad.tar.gz")
resp = predictor.predict(
  passage="The Matrix is a 1999 science fiction action film written and directed by "
          "The Wachowskis, starring Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, "
          "Hugo Weaving, and Joe Pantoliano.",
  question="Who stars in The Matrix?"
)

print("\n".join(resp))

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


100%|██████████| 46175392/46175392 [00:05<00:00, 7810723.56B/s] 


passage_question_attention
span_start_logits
span_start_probs
span_end_logits
span_end_probs
best_span
best_span_str
question_tokens
passage_tokens


In [3]:
resp['best_span_str']

'Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Joe Pantoliano'

From the command line:

In [4]:
%%bash
TEXT='{"passage": "The Matrix is a 1999 science fiction action film written and directed by 
The Wachowskis, starring Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, 
and Joe Pantoliano.", 
"question": "Who stars in The Matrix?"}' 
URL_BASE='https://s3-us-west-2.amazonaws.com/allennlp/models'
MODEL_URL=$URL_BASE'/bidaf-model-2017.09.15-charpad.tar.gz'
RESP=$(echo $TEXT | allennlp predict $MODEL_URL - 2>/dev/null)
echo $RESP | sed 's/.*prediction: //' | jq .best_span_str

"Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Joe Pantoliano"


### Textual Entailment

We skip the `bash` versions for the rest of these examples.  They can be found at the AllenNLP demo web page linked above.  The [live demo](http://demo.allennlp.org/textual-entailment) visualizes results in a nice way.

In [5]:
from functools import partial
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/"
                                "decomposable-attention-elmo-2018.02.19.tar.gz")

follows = partial(predictor.predict, 
                  premise="Two women are sitting on a blanket near some "
                          "rocks talking about politics.")

print(predictor.__class__.__name__)

100%|██████████| 697657697/697657697 [01:03<00:00, 11046940.98B/s]


DecomposableAttentionPredictor


In [6]:
resp = follows(hypothesis="The pair of women have political talks.")

print("\n".join(resp))

label_logits
label_probs
h2p_attention
p2h_attention
premise_tokens
hypothesis_tokens


In [7]:
# The listed target descriptions are built into the DecomposableAttentionPredictor
for entailment, prob in zip(
    ['entailment', 'contradiction', 'neutral'], resp['label_probs']):
    print(f"{entailment:>13}: {prob*100:5.2f}%")

   entailment: 63.50%
contradiction:  0.38%
      neutral: 36.12%


In [8]:
resp = follows(hypothesis="Two women are wandering along the shore drinking iced tea.")

for entailment, prob in zip(
    ['entailment', 'contradiction', 'neutral'], resp['label_probs']):
    print(f"{entailment:>13}: {prob*100:5.2f}%")

   entailment:  0.00%
contradiction: 99.95%
      neutral:  0.05%


### Semantic Role Labeling

In [9]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/"
                                "srl-model-2018.05.25.tar.gz")
resp = predictor.predict(
  sentence="Did Uriah honestly think he could beat the game in under three hours?"
)

100%|██████████| 730942854/730942854 [01:02<00:00, 11784953.35B/s]


In [10]:
for verb in resp['verbs']:
    print(verb['verb'].upper(), '\n ', verb['description'], '\n')

DID 
  [V: Did] Uriah honestly think he could beat the game in under three hours ? 

THINK 
  Did [ARG0: Uriah] [ARGM-MNR: honestly] [V: think] [ARG1: he could beat the game in under three hours] ? 

COULD 
  Did Uriah honestly think he [V: could] beat the game in under three hours ? 

BEAT 
  Did Uriah honestly think [ARG0: he] [ARGM-MOD: could] [V: beat] [ARG1: the game] [ARGM-TMP: in under three hours] ? 



### Named Entity Recognition

In [11]:
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/"
                                "ner-model-2018.12.18.tar.gz")
resp = predictor.predict(
  sentence="AllenNLP is a PyTorch-based natural language processing library developed "
           "at the Allen Institute for Artificial Intelligence in Seattle."
)

100%|██████████| 711306412/711306412 [02:38<00:00, 4480962.91B/s] 


In [12]:
phrase = []
for tag, word in zip(resp['tags'], resp['words']):
    if tag.startswith('U-'):
        print(tag.replace('U-', ''), word)
    elif tag == 'O':
        print('   ', word)
    elif tag.startswith('B-'):
        phrase.append(word)
    elif tag.startswith('I-'):
        phrase.append(word)
    elif tag.startswith('L-'):
        phrase.append(word)
        print(tag.replace('L-', ''), ' '.join(phrase))
        phrase = []
    else:
        print("PROBLEM")

ORG AllenNLP
    is
    a
ORG PyTorch
    -
    based
    natural
    language
    processing
    library
    developed
    at
    the
ORG Allen Institute for Artificial Intelligence
    in
LOC Seattle
    .


### Other demos

There are several additional interesing demos provided with the [AllenNLP model examples](https://allennlp.org/models).  Most likely more will be added over time. 

### Cleanup large cached models

In [13]:
!rm ~/.allennlp/cache/*
!rm -rf /tmp/*

## Next Lesson

**Enhancing an Image Classifier**: Libraries built on top of PyTorch offer very powerful tools for natural language processing.  Next we will return to image classification, and build on pretrained models as a demonstration of re-use neural networks.

<a href="ImageClassifier.ipynb"><img src="img/open-notebook.png" align="left"/></a>